In [1]:
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
from typing import List, Dict, Tuple, Optional
import pandas as pd
from openpyxl import load_workbook
from geopy.geocoders import Nominatim
import numpy as np
import os
import math
import polars as pl
import polars.selectors as cs

In [2]:
from openpyxl import load_workbook

In [3]:
df_destinations = pd.read_excel('../data/destinations.xlsx')
df_destinations

,Country,Place,Type,Lat,Lon
0,Egypte,Hurghada,zon,27.222556,33.830706
1,Egypte,Marsa Alam,zon,25.069992,34.893817
2,Egypte,Sharm el Sheikh,zon,27.864442,34.295447
3,Italië,Rome,stad,41.893320,12.482932
4,Nederland,Ede,NaN,52.071683,5.745511
5,Spanje,Barcelona,stad,41.382894,2.177432
6,Spanje,Palma de Mallorca,zon,39.569582,2.650075
7,Egypte,Cairo,stad,30.044388,31.235726
8,Egypte,Luxor,zon,25.702096,32.647186
9,Turkije,Alanya,zon,36.886573,30.703024


In [4]:
def create_dict_destinations(df_destinations):
    cols = list(df_destinations.columns)
    dict_destinations = {}
    for i,v in df_destinations.iterrows():
        country = v['Country'].lower().replace(' ','-')
        place = v['Place'].lower().replace(' ','-')
        lat = v['Lat']
        if country not in dict_destinations:
            dict_destinations[country]={}
        dict_destinations[country][place] = {col:v[col] for col in cols}
        
        # if lat is None or np.isnan(lat):
        #     dict_latlon = add_latlon(country=v['Country'],place=v['Place'])
        #     dict_destinations[country][place].update(dict_latlon)
    return(dict_destinations)

dict_destinations = create_dict_destinations(df_destinations)
dict_destinations['egypte']['hurghada']

{'Country': 'Egypte',
 'Place': 'Hurghada',
 'Type': 'zon',
 'Lat': 27.222556,
 'Lon': 33.8307062}

In [5]:
# import os
# import pandas as pd

# country='egypte'
# place='sharm-el-sheikh'
# fp_daily=os.path.join('../data/raw',country,place,'weather_daily.csv')

# df_daily=pd.read_csv(fp_daily,index_col='time')
# df_daily

In [6]:
# import os
# import polars as pl
# import pandas as pd
# from datetime import date

# country='egypte'
# place='sharm-el-sheikh'
# # country='spanje'
# # place='barcelona'
# country='nederland'
# place='ede'

# def load_filter_add_daily(country,place,last_n_years=7,folder_data_raw='./data/raw'):
#     # Load daily data:
#     fp_cp_d = os.path.join(folder_data_raw,country,place,'weather_daily.csv')
#     df_cp_d = pl.read_csv(fp_cp_d,try_parse_dates=True).select(
#         ['time','tavg','tmin','tmax','prcp','wspd','pres']
#     ).cast(
#         {'time':pl.Date,
#          'tavg':pl.Float64,'tmin':pl.Float64,'tmax':pl.Float64,
#          'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64}
#     ).filter(
#         (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
#     ).with_columns(
#         # pl.col('time').dt.strftime('%m-%d').alias('md'),
#         pl.col('time').dt.year().alias('y'),
#         # pl.col('time').dt.strftime('%m').alias('m_'),
#         pl.col('time').dt.month().alias('m'),
#         pl.when(pl.col("time").dt.day() > 20)
#             .then(3)
#             .when(pl.col("time").dt.day() > 10)
#             .then(2)
#             .otherwise(1)
#             .alias("m3"),
#         pl.col('time').dt.day().alias('d'),
#         pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
#     ).cast({'m3':pl.Int8,'rain':pl.Int8})
#     return(df_cp_d)

# df_daily = load_filter_add_daily(country,place,last_n_years=7,folder_data_raw='../data/raw')
# df_daily

In [7]:
def get_weather(lat,lon,start,end,frequency='daily'):
    if frequency=='daily':
        data = Daily(Point(lat,lon),start,end)
    elif frequency=='hourly':
        data = Hourly(Point(lat,lon),start,end)
    data = data.fetch()
    data = data[data.index<(datetime.now()-timedelta(days=1))]
    return data

In [8]:
# import math
# from datetime import datetime,timedelta

country='egypte'
place='sharm-el-sheikh'

def download_add_daily(country,place,dict_destinations,last_n_years=7):
    dict_cp = dict_destinations[country][place]
    lat,lon = dict_cp['Lat'],dict_cp['Lon']
    days = math.ceil(last_n_years*365.25)
    start = (datetime.today()-timedelta(days=days))
    end = datetime.today()
    data_raw = get_weather(lat=lat,lon=lon,start=start,end=end,frequency='daily')
    df_cp_d = pl.from_pandas(data_raw,include_index=True).select(
        ['time','tavg','tmin','tmax','prcp','wspd','pres']
    ).cast(
        {'time':pl.Date,
         'tavg':pl.Float64,'tmin':pl.Float64,'tmax':pl.Float64,
         'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64}
    ).filter(
        (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
    ).with_columns(
        # pl.col('time').dt.strftime('%m-%d').alias('md'),
        pl.col('time').dt.year().alias('y'),
        # pl.col('time').dt.strftime('%m').alias('m_'),
        pl.col('time').dt.month().alias('m'),
        pl.when(pl.col("time").dt.day() > 20)
            .then(3)
            .when(pl.col("time").dt.day() > 10)
            .then(2)
            .otherwise(1)
            .alias("m3"),
        pl.col('time').dt.day().alias('d'),
        pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
    ).cast({'m3':pl.Int8,'rain':pl.Int8})
    return(df_cp_d)

df_daily = download_add_daily(country,place,dict_destinations,last_n_years=7)
df_daily

time,tavg,tmin,tmax,prcp,wspd,pres,y,m,m3,d,rain
date,f64,f64,f64,f64,f64,f64,i32,i8,i8,i8,i8
2018-07-21,32.1,28.0,36.8,null,15.5,1006.7,2018,7,3,21,null
2018-07-22,31.6,27.3,36.1,null,16.4,1006.3,2018,7,3,22,null
2018-07-23,33.0,28.5,37.9,null,14.9,1005.3,2018,7,3,23,null
2018-07-24,33.8,28.6,38.6,null,14.5,1005.5,2018,7,3,24,null
2018-07-25,34.9,29.3,40.1,null,11.4,1006.6,2018,7,3,25,null
…,…,…,…,…,…,…,…,…,…,…,…
2025-01-01,12.6,8.9,16.4,0.0,10.7,1024.7,2025,1,1,1,0
2025-01-02,12.7,8.0,17.2,0.0,7.5,1024.7,2025,1,1,2,0
2025-01-03,13.7,8.3,18.7,0.0,4.0,1021.9,2025,1,1,3,0


In [9]:
country='egypte'
place='sharm-el-sheikh'
# country='spanje'
# place='barcelona'
# country='nederland'
# place='ede'

# from datetime import datetime,timedelta
# import math
# from meteostat import Point, Daily, Hourly

def download_add_hourly(country,place,dict_destinations,last_n_years=7):
    dict_cp = dict_destinations[country][place]
    lat,lon = dict_cp['Lat'],dict_cp['Lon']
    days = math.ceil(last_n_years*365.25)
    start = (datetime.today()-timedelta(days=days))
    end = datetime.today()
    data_raw = get_weather(lat=lat,lon=lon,start=start,end=end,frequency='hourly')
    df_cp_h = pl.from_pandas(data_raw,include_index=True).select(
        ['time','temp','rhum','prcp','wspd','pres','coco']
    ).cast(
        {'time':pl.Datetime,
         'temp':pl.Float64,'rhum':pl.Float64,
         'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64,
         'coco':pl.Float64
        }
    ).with_columns(
        pl.col('time').dt.year().alias('y'),
        pl.col('time').dt.month().alias('m'),
        pl.col('time').dt.day().alias('d'),
        pl.col('time').dt.hour().alias('h'),
        pl.when(pl.col("time").dt.day() > 20)
            .then(3)
            .when(pl.col("time").dt.day() > 10)
            .then(2)
            .otherwise(1)
            .alias("m3"),
        pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
    ).cast({'m3':pl.Int8,'rain':pl.Int8})
    return (df_cp_h)

df_hourly = download_add_hourly(country,place,dict_destinations,last_n_years=7)
df_hourly

time,temp,rhum,prcp,wspd,pres,coco,y,m,d,h,m3,rain
datetime[μs],f64,f64,f64,f64,f64,f64,i32,i8,i8,i8,i8,i8
2018-01-28 18:00:00,16.6,51.0,null,16.7,1022.8,1.0,2018,1,28,18,3,null
2018-01-28 19:00:00,null,null,null,null,null,null,2018,1,28,19,3,null
2018-01-28 20:00:00,null,null,null,null,null,null,2018,1,28,20,3,null
2018-01-28 21:00:00,14.6,57.0,null,11.1,1022.5,1.0,2018,1,28,21,3,null
2018-01-28 22:00:00,null,null,null,null,null,null,2018,1,28,22,3,null
…,…,…,…,…,…,…,…,…,…,…,…,…
2025-01-05 10:00:00,20.6,32.0,0.0,13.0,1009.1,1.0,2025,1,5,10,1,0
2025-01-05 11:00:00,21.3,32.0,0.0,16.7,1008.4,1.0,2025,1,5,11,1,0
2025-01-05 12:00:00,21.9,33.0,0.0,18.5,1007.7,1.0,2025,1,5,12,1,0


In [10]:
# import polars.selectors as cs

country='egypte'
place='sharm-el-sheikh'
# country='spanje'
# place='barcelona'
# country='nederland'
# place='ede'


def preprocess_daily(country,place,dict_destinations,last_n_years=7):
    """Create dict with multiple aggregate df's: mm3d, mm3, m
    All averages (except rain) are the averages of the upper and lower bound, defined as 90% quantile
    rain_pct, rain_sum is taken over all days (i.e. not deleting extreme points)
    """
    # df_cp_d = load_filter_add_daily(country=country,place=place,last_n_years=last_n_years,folder_data_raw=folder_data_raw)
    df_cp_d = download_add_daily(country,place,dict_destinations,last_n_years=last_n_years)
    cols_d = ['tmin','tavg','tmax','prcp','wspd']
    df_ub_d = df_cp_d.group_by(['m','m3','d']).agg(
        pl.col(c).quantile(0.9,'lower').alias(str(c)+'_ub') for c in cols_d
    )
    df_lb_d = df_cp_d.group_by(['m','m3','d']).agg(
        pl.col(c).quantile(0.1,'higher').alias(str(c)+'_lb') for c in cols_d
    )
    df_rain = df_cp_d.group_by(['m','m3','d']).agg(
        pl.col('rain').mean().round(2).alias('rain_pct')
    )    
    df_mm3d = df_lb_d.join(
        df_ub_d,on=['m','m3','d'],how='full', coalesce=True
    ).join(
        df_rain,on=['m','m3','d'],how='full', coalesce=True
    ).with_columns(
        ((pl.col(str(c)+'_ub')+pl.col(str(c)+'_lb'))/2).round(1).alias(str(c)+'_avg') for c in cols_d
        ).select(
            cs.by_dtype([pl.Int8]),cs.matches('_avg$'),cs.matches('_pct$')
        ).sort(['m','d'])

    df_m = df_mm3d.group_by(['m']).agg(
        cs.matches('^tmin|^tmax|^tavg|^wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        
        cs.matches('rain_pct').sum().round(0).alias('rain_sum'),
        cs.matches('rain_pct').mean().round(2).alias('rain_pct'),
        cs.matches('prcp').sum().round(0).name.map(lambda c:c.replace('_avg','_sum'))
    ).sort(['m'])

    df_mm3 = df_mm3d.group_by(['m','m3']).agg(
        cs.matches('^tmin|^tmax|^tavg|^wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        cs.matches('rain_pct').sum().round(0).alias('rain_sum'),
        cs.matches('rain_pct').mean().round(2).alias('rain_pct'),
        cs.matches('prcp').sum().round(0).name.map(lambda c:c.replace('_avg','_sum'))
    ).sort(['m','m3'])
    return({'df_mm3d':df_mm3d,
            'df_mm3':df_mm3,
            'df_m':df_m
           })
    

df_daily_processed = preprocess_daily(country,place,dict_destinations,last_n_years=7)
# df_daily_processed['df_mm3d'].head(20).tail(10)
# # df_daily_processed['df_m'].head(12)
# df_daily_processed['df_mm3'].head(12)
print(df_daily_processed)

{'df_mm3d': shape: (366, 9)
┌─────┬─────┬─────┬──────────┬───┬──────────┬──────────┬──────────┬──────────┐
│ m   ┆ m3  ┆ d   ┆ tmin_avg ┆ … ┆ tmax_avg ┆ prcp_avg ┆ wspd_avg ┆ rain_pct │
│ --- ┆ --- ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ i8  ┆ i8  ┆ i8  ┆ f64      ┆   ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞═════╪═════╪═════╪══════════╪═══╪══════════╪══════════╪══════════╪══════════╡
│ 1   ┆ 1   ┆ 1   ┆ 10.4     ┆ … ┆ 18.8     ┆ 0.0      ┆ 6.6      ┆ 0.25     │
│ 1   ┆ 1   ┆ 2   ┆ 11.2     ┆ … ┆ 19.0     ┆ 0.0      ┆ 7.2      ┆ 0.2      │
│ 1   ┆ 1   ┆ 3   ┆ 10.6     ┆ … ┆ 19.3     ┆ 0.1      ┆ 6.5      ┆ 0.4      │
│ 1   ┆ 1   ┆ 4   ┆ 10.1     ┆ … ┆ 19.8     ┆ 0.0      ┆ 6.5      ┆ 0.0      │
│ 1   ┆ 1   ┆ 5   ┆ 10.3     ┆ … ┆ 20.0     ┆ 0.0      ┆ 7.0      ┆ 0.0      │
│ …   ┆ …   ┆ …   ┆ …        ┆ … ┆ …        ┆ …        ┆ …        ┆ …        │
│ 12  ┆ 3   ┆ 27  ┆ 11.9     ┆ … ┆ 19.6     ┆ 0.0      ┆ 10.9     ┆ 0.0      │
│ 12  ┆ 3   ┆ 28  ┆ 11.2

In [11]:

# country='egypte'
# place='sharm-el-sheikh'
# # country='spanje'
# # place='barcelona'
# # country='nederland'
# # place='ede'

# def preprocess_csv(country,place,last_n_years=7,folder_data_raw='./data/raw',folder_data_processed='./data/processed'):
#     dfs_daily = preprocess_daily(country=country,place=place,last_n_years=last_n_years,folder_data_raw=folder_data_raw)
#     for df in dfs_daily:
#         dir_cp = os.path.join(folder_data_processed,country,place)
#         os.makedirs(dir_cp,exist_ok=True)
#         fp_csv = os.path.join(dir_cp,df+'.csv')
#         dfs_daily[df].write_csv(fp_csv,separator=',')

# preprocess_csv(country,place,last_n_years=7,folder_data_raw='../data/raw',folder_data_processed='../data/processed')

In [13]:
def dictdfs_write_csv(dict_dfs,country,place=None,folder_data_processed='./data/processed'):
    dir_csv = os.path.join(folder_data_processed,'' if (country is None) else country,'' if (place is None) else place )
    for df in dict_dfs:
        os.makedirs(dir_csv,exist_ok=True)
        fp_csv = os.path.join(dir_csv,df+'.csv')
        dict_dfs[df].write_csv(fp_csv,separator=',')

country='egypte'
place='sharm-el-sheikh'
folder_data_raw='../data/raw'
folder_data_processed='../data/processed'
dfs_daily = preprocess_daily(country,place,dict_destinations,last_n_years=7)

# place=''
dictdfs_write_csv(dict_dfs=dfs_daily,country=country,place=place,folder_data_processed=folder_data_processed)

In [11]:
country='egypte'
place='sharm-el-sheikh'
# country='spanje'
# place='barcelona'
# country='nederland'
# place='ede'


def preprocess_hourly(country,place,dict_destinations,last_n_years=5):
    df_cp_h = download_add_hourly(country,place,dict_destinations,last_n_years=last_n_years)
    cols_h = ['temp','rhum','prcp','wspd'] # and coco
    df_ub_h = df_cp_h.group_by(['m','m3','d','h']).agg(
        pl.col(c).quantile(0.9,'lower').alias(str(c)+'_ub') for c in cols_h
    )
    df_lb_h = df_cp_h.group_by(['m','m3','d','h']).agg(
        pl.col(c).quantile(0.1,'higher').alias(str(c)+'_lb') for c in cols_h
    )
    df_rain = df_cp_h.group_by(['m','m3','d','h']).agg(
        pl.col('rain').mean().round(2).alias('rain_pct')
    )  
    
    # df_mm3dh used for other calcs, not part of output:
    df_mm3dh = df_lb_h.join(
        df_ub_h,on=['m','m3','d','h'],how='full',coalesce=True
    ).join(
        df_rain,on=['m','m3','d','h'],how='full', coalesce=True
    ).with_columns(
        ((pl.col(str(c)+'_ub')+pl.col(str(c)+'_lb'))/2).round(1).alias(str(c)+'_avg') for c in cols_h
        ).select(
        cs.by_dtype([pl.Int8]),pl.col('temp_lb'),pl.col('temp_ub'),cs.matches('_avg$'),cs.matches('_pct$')
    )
    df_mm3h = df_mm3dh.group_by(['m','m3','h']).agg(
        cs.matches('temp|rhum|wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        cs.matches('rain_pct').mean().round(2).alias('rain_pct'),
    ).sort(['m','m3','h'])
    df_mh = df_mm3dh.group_by(['m','h']).agg(
        cs.matches('temp|rhum|wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        cs.matches('rain_pct').mean().round(2).alias('rain_pct')
    ).sort(['m','h'])

    # coco calculations, df_mm3dhc used for other calcs
    df_mm3dhc = df_cp_h.group_by(
        ['m','m3','d','h','coco']
    ).len().cast({'len':pl.Int8}).filter(
        pl.col('coco').is_not_null()
    ).filter(
        pl.col('coco')>0
    ).pivot(
        'coco',index=['m','m3','d','h'],values='len'
    ).fill_null(0).unpivot(
        cs.exclude('m','m3','d','h'),index=['m','m3','d','h']
    ).cast({'variable':pl.Float64}).with_columns(
        (pl.col('value')/pl.col('value').sum().over(['m','m3','d','h'])).round(3)
    ).rename({'variable':'coco','value':'coco_pct'})

    # df_mhc = df_mm3dhc.group_by(['m','h','coco']).agg(pl.col('coco_pct').mean().round(3))
    df_mc = df_mm3dhc.group_by(['m','coco']).agg(pl.col('coco_pct').mean().round(3))
    df_mm3c = df_mm3dhc.group_by(['m','m3','coco']).agg(pl.col('coco_pct').mean().round(3))
    
    return({
        'df_mc':df_mc,
        'df_mm3c':df_mm3c,
        'df_mm3h':df_mm3h,
        'df_mh':df_mh
    })

dfs_hourly = preprocess_hourly(country,place,dict_destinations,last_n_years=5)
# df_h['df_mhc'].sort(['m','h','coco_pct','coco'],descending=[False,False,True,False])
dfs_hourly['df_mm3c'].sort(['m','m3','coco_pct','coco'],descending=[False,False,True,False])
# df_h


m,m3,coco,coco_pct
i8,i8,f64,f64
1,1,1.0,0.724
1,1,2.0,0.211
1,1,18.0,0.03
1,1,3.0,0.024
1,1,4.0,0.005
…,…,…,…
12,3,3.0,0.052
12,3,18.0,0.034
12,3,7.0,0.002


In [59]:
dictdfs_write_csv(dfs_hourly,country,place,'../data/processed')

In [399]:
import os
import polars as pl
import pandas as pd
from datetime import date

country='egypte'
place='sharm-el-sheikh'
# country='spanje'
# place='barcelona'
country='nederland'
place='ede'

def load_filter_add_cp(country,place,last_n_years=5,folder_data_raw='./data/raw'):
    # Load daily data:
    fp_cp_d = os.path.join(folder_data_raw,country,place,'weather_daily.csv')
    df_cp_d = pl.read_csv(fp_cp_d,try_parse_dates=True).select(
        ['time','tavg','tmin','tmax','prcp','wspd','pres']
    ).cast(
        {'time':pl.Date,
         'tavg':pl.Float64,'tmin':pl.Float64,'tmax':pl.Float64,
         'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64}
    ).filter(
        (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
    ).with_columns(
        # pl.col('time').dt.strftime('%m-%d').alias('md'),
        pl.col('time').dt.year().alias('y'),
        # pl.col('time').dt.strftime('%m').alias('m_'),
        pl.col('time').dt.month().alias('m'),
        pl.when(pl.col("time").dt.day() > 20)
            .then(3)
            .when(pl.col("time").dt.day() > 10)
            .then(2)
            .otherwise(1)
            .alias("m3"),
            # .alias("m3_"),
        pl.col('time').dt.day().alias('d'),
        pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
    ).cast({'m3':pl.Int8,'rain':pl.Int8})
    # ).with_columns(
    #     pl.concat_str(['m_','m3_'],separator='-').alias('m3')
    # ).drop(['m_','m3_'])
    # Load hourly data:
    fp_cp_h = os.path.join(folder_data_raw,country,place,'weather_hourly.csv')
    df_cp_h = pl.read_csv(fp_cp_h,try_parse_dates=True).select(
        ['time','temp','rhum','prcp','wspd','pres','coco']
    ).cast(
        {'time':pl.Datetime,
         'temp':pl.Float64,'rhum':pl.Float64,
         'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64,
         'coco':pl.Float64
        }
    ).filter(
        (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
    ).with_columns(
        # pl.col('time').dt.strftime('%m-%d').alias('md'),
        # pl.col('time').dt.strftime('%m').alias('m_'),
        pl.col('time').dt.year().alias('y'),
        pl.col('time').dt.month().alias('m'),
        pl.col('time').dt.day().alias('d'),
        pl.col('time').dt.hour().alias('h'),
        pl.when(pl.col("time").dt.day() > 20)
            .then(3)
            .when(pl.col("time").dt.day() > 10)
            .then(2)
            .otherwise(1)
            .alias("m3"),
        pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
        # pl.col('time').dt.day().alias('d'),
        
    # ).with_columns(
    #     pl.concat_str(['m_','m3_'],separator='-').alias('m3')
    ).cast({'m3':pl.Int8,'rain':pl.Int8})#.drop(['m_','m3_']).cast({str(c):pl.Int8} for c in ['y','m','m3','h','rain'])
    #
    return({'df_cp_h':df_cp_h,'df_cp_d':df_cp_d})

data_cp = load_filter_add_cp(country,place,last_n_years=5,folder_data_raw='../data/raw')

df_cp_d = data_cp['df_cp_d']
df_cp_h = data_cp['df_cp_h']
df_cp_h.sort(['m','d','y']).tail(10)

time,temp,rhum,prcp,wspd,pres,coco,y,m,d,h,m3,rain
datetime[μs],f64,f64,f64,f64,f64,f64,i32,i8,i8,i8,i8,i8
2023-12-31 15:00:00,8.6,74.0,0.0,21.6,995.7,7.0,2023,12,31,15,3,0
2023-12-31 16:00:00,8.5,74.0,0.0,21.6,995.6,7.0,2023,12,31,16,3,0
2023-12-31 17:00:00,8.7,73.0,0.5,25.2,995.5,7.0,2023,12,31,17,3,1
2023-12-31 18:00:00,8.7,74.0,0.3,25.2,995.8,7.0,2023,12,31,18,3,1
2023-12-31 19:00:00,8.0,81.0,0.0,21.6,996.1,7.0,2023,12,31,19,3,0
2023-12-31 20:00:00,8.1,79.0,0.0,25.2,996.2,7.0,2023,12,31,20,3,0
2023-12-31 21:00:00,8.2,79.0,0.0,25.2,996.5,7.0,2023,12,31,21,3,0
2023-12-31 22:00:00,8.5,75.0,0.0,32.4,996.9,7.0,2023,12,31,22,3,0
2023-12-31 23:00:00,8.2,76.0,0.2,28.8,997.2,7.0,2023,12,31,23,3,1


In [440]:
country='egypte'
place='sharm-el-sheikh'
# country='spanje'
# place='barcelona'
country='nederland'
place='ede'

import polars.selectors as cs

def create_output_cp(country,place,last_n_years=5,folder_data_raw='./data/raw'):
    data_cp = load_filter_add_cp(country,place,last_n_years=last_n_years,folder_data_raw=folder_data_raw)
    df_cp_d = data_cp['df_cp_d']
    df_cp_h = data_cp['df_cp_h']

    cols_d = ['tmin','tavg','tmax','prcp','rain','wspd']
    cols_h = ['temp','rhum','prcp','rain','wspd'] # and coco
    
    df_ub_d = df_cp_d.group_by(['m','m3','d']).agg(
        pl.col(c).quantile(0.9,'lower').alias(str(c)+'_ub') for c in cols_d
    )
    df_lb_d = df_cp_d.group_by(['m','m3','d']).agg(
        pl.col(c).quantile(0.1,'higher').alias(str(c)+'_lb') for c in cols_d
    )
    df_ub_h = df_cp_h.group_by(['m','m3','d','h']).agg(
        pl.col(c).quantile(0.9,'lower').alias(str(c)+'_ub') for c in cols_h
    )
    df_lb_h = df_cp_h.group_by(['m','m3','d','h']).agg(
        pl.col(c).quantile(0.1,'higher').alias(str(c)+'_lb') for c in cols_h
    )
    # Aggregates based on hourly data:
    # df_mm3dh used for other calcs, not part of output:
    df_mm3dh = df_lb_h.join(df_ub_h,on=['m','m3','d','h'],how='full',coalesce=True).with_columns(
        ((pl.col(str(c)+'_ub')+pl.col(str(c)+'_lb'))/2).round(1).alias(str(c)+'_avg') for c in cols_h
        ).select(
        cs.by_dtype([pl.Int8]),pl.col('temp_lb'),pl.col('temp_ub'),cs.matches('avg')
    )
    df_mm3h = df_mm3dh.group_by(['m','m3','h']).agg(
        cs.matches('temp|rhum|wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med'))
    ).sort(['m','m3','h'])
    df_mh = df_mm3dh.group_by(['m','h']).agg(
        cs.matches('temp|rhum|wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med'))
    ).sort(['m','h'])
    
    # Aggregates based on daily data:
    df_mm3d = df_lb_d.join(df_ub_d,on=['m','m3','d'],how='full', coalesce=True).with_columns(
        ((pl.col(str(c)+'_ub')+pl.col(str(c)+'_lb'))/2).round(1).alias(str(c)+'_avg') for c in cols_d
        ).select(
            cs.by_dtype([pl.Int8]),cs.matches('_avg$')
        ).sort(['m','d'])#.select(cs.by_dtype([pl.Int8]),cs.starts_with('tmin'))

    df_m = df_mm3d.group_by(['m']).agg(
        cs.matches('t|wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        cs.matches('rain|prcp').sum().round(0).name.map(lambda c:c.replace('_avg','_sum'))
    ).sort(['m'])

    df_mm3 = df_mm3d.group_by(['m','m3']).agg(
        cs.matches('t|wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        cs.matches('rain|prcp').sum().round(0).name.map(lambda c:c.replace('_avg','_sum'))
    ).sort(['m','m3'])
    
    
    return({
        'df_mm3h':df_mm3h,
        'df_mh':df_mh,
        # 'df_mm3dh':df_mm3dh,
        'df_mm3d':df_mm3d,
        'df_mm3':df_mm3,
        'df_m':df_m,
    })


data_output = create_output_cp(country,place,last_n_years=5,folder_data_raw='../data/raw')
# data_output['df_mm3d']
data_output['df_m']#.sort(['m','h'])#.sort(['m','m3','h','d']).select(cs.by_dtype([pl.Int8]),pl.col('temp_lb'),pl.col('temp_ub'),cs.matches('avg'))

m,tmin_med,tavg_med,tmax_med,wspd_med,prcp_sum,rain_sum
i8,f64,f64,f64,f64,f64,f64
1,2.0,4.1,6.4,16.4,96.0,19.0
2,2.6,5.8,8.9,17.5,89.0,21.0
3,2.2,7.4,11.4,14.2,81.0,23.0
4,3.6,9.4,14.3,13.9,99.0,22.0
5,8.4,13.5,19.2,11.8,97.0,23.0
…,…,…,…,…,…,…
8,13.6,18.4,23.6,10.1,29.0,17.0
9,11.0,15.4,20.9,10.4,29.0,15.0
10,8.8,12.0,15.1,13.2,44.0,22.0


In [441]:
data_output

{'df_mm3h': shape: (864, 8)
 ┌─────┬─────┬─────┬─────────┬─────────┬──────────┬──────────┬──────────┐
 │ m   ┆ m3  ┆ h   ┆ temp_lb ┆ temp_ub ┆ temp_med ┆ rhum_med ┆ wspd_med │
 │ --- ┆ --- ┆ --- ┆ ---     ┆ ---     ┆ ---      ┆ ---      ┆ ---      │
 │ i8  ┆ i8  ┆ i8  ┆ f64     ┆ f64     ┆ f64      ┆ f64      ┆ f64      │
 ╞═════╪═════╪═════╪═════════╪═════════╪══════════╪══════════╪══════════╡
 │ 1   ┆ 1   ┆ 0   ┆ 1.7     ┆ 7.1     ┆ 4.3      ┆ 94.5     ┆ 14.4     │
 │ 1   ┆ 1   ┆ 1   ┆ 1.3     ┆ 7.1     ┆ 4.3      ┆ 94.8     ┆ 15.3     │
 │ 1   ┆ 1   ┆ 2   ┆ 1.4     ┆ 7.5     ┆ 4.4      ┆ 93.8     ┆ 16.2     │
 │ 1   ┆ 1   ┆ 3   ┆ 1.2     ┆ 7.8     ┆ 4.4      ┆ 94.3     ┆ 17.1     │
 │ 1   ┆ 1   ┆ 4   ┆ 1.2     ┆ 8.0     ┆ 4.5      ┆ 95.0     ┆ 17.1     │
 │ …   ┆ …   ┆ …   ┆ …       ┆ …       ┆ …        ┆ …        ┆ …        │
 │ 12  ┆ 3   ┆ 19  ┆ 5.5     ┆ 8.5     ┆ 6.7      ┆ 95.0     ┆ 19.8     │
 │ 12  ┆ 3   ┆ 20  ┆ 5.5     ┆ 8.5     ┆ 6.4      ┆ 95.5     ┆ 19.8     │
 │ 12  ┆ 3

In [332]:
country='egypte'
place='sharm-el-sheikh'
country='spanje'
place='barcelona'
country='nederland'
place='ede'

def create_output_cp(country,place,last_n_years=5,folder_data_raw='./data/raw'):
    data_cp = load_filter_add_cp(country,place,last_n_years=last_n_years,folder_data_raw=folder_data_raw)
    df_cp_d = data_cp['df_cp_d']
    df_cp_h = data_cp['df_cp_h']

    # <country>/<place>/<month> + filter/month : 
    # - line+area chart temperatures
    # - bar chart rain pct
    # - bar chart rain avg
    df_md = df_cp_d.group_by(['md','m3','m','d']).agg(
        pl.col('tmin').min().round(1).alias('tmin_min'),
        pl.col('tmin').median().round(1).alias('tmin_med'),
        pl.col('tavg').median().round(1).alias('tavg_med'),
        pl.col('tmax').median().round(1).alias('tmax_med'),
        pl.col('tmax').max().round(1).alias('tmax_max'),
        pl.col('prcp').median().round(1).alias('prcp_med'),
        pl.col('rain').median().round(1).alias('rain_med')
        # (pl.col('rain').sum()/pl.count('rain')).round(2).alias('rain_pct'),
        # (pl.col('prcp').sum()/pl.count('rain')).round().alias('rain_avg')
    ).sort('md').sort(['m','d'])

    # <country>/<place>/<month> + filter/month :
    # - stat temperatures
    # <country>/<place>:
    # - line+area chart temperatures entire year
    df_m3 = df_md.group_by(['m3','m']).agg(
        pl.col('tmin_min').median().round(1).alias('tmin_min_med'),
        pl.col('tmin_med').median().round(1).alias('tmin_med_med'),
        pl.col('tavg_med').median().round(1).alias('tavg_med_med'),
        pl.col('tmax_med').median().round(1).alias('tmax_med_med'),
        pl.col('tmax_max').median().round(1).alias('tmax_max_med'),
        pl.col('prcp_med').median().round(1).alias('prcp_med_med'),
        pl.col('rain_med').median().round(2).alias('rain_med_avg')
        # pl.col('rain_pct').mean().alias('rain_pct_avg'),
        # pl.col('rain_avg').sum().alias('rain_mm')
    ).sort('m3')

    
    df_m = df_md.group_by(['m']).agg(
        pl.col('tmin_min').median().round(1).alias('tmin_min_med'),
        pl.col('tmin_med').median().round(1).alias('tmin_med_med'),
        pl.col('tavg_med').median().round(1).alias('tavg_med_med'),
        pl.col('tmax_med').median().round(1).alias('tmax_med_med'),
        pl.col('tmax_max').median().round(1).alias('tmax_max_med'),
        pl.col('prcp_med').median().round(1).alias('prcp_med_med'),
        pl.col('rain_med').median().round(2).alias('rain_med_avg')
        # pl.col('rain_avg').sum().alias('rain_mm'),
        # pl.col('rain_pct').sum().alias('rain_days')
    ).sort('m')

    df_mh = df_cp_h.group_by(['y','m','h']).agg(
        pl.col('temp').min().round(1).alias('temp_min'),
        pl.col('temp').median().round(1).alias('temp_med'),
        pl.col('temp').max().round(1).alias('temp_max')
    ).group_by(['m','h']).agg(
        pl.col('temp_min').min().round(1).alias('temp_min_min'),
        pl.col('temp_min').median().round(1).alias('temp_min_med'),
        pl.col('temp_med').median().round(1).alias('temp_med_med'),
        pl.col('temp_max').median().round(1).alias('temp_max_med'),
        pl.col('temp_max').max().round(1).alias('temp_max_max')
    ).sort(['m','h'])

    df_m3h = df_cp_h.group_by(['y','m3','h']).agg(
        pl.col('temp').min().round(1).alias('temp_min'),
        pl.col('temp').median().round(1).alias('temp_med'),
        pl.col('temp').max().round(1).alias('temp_max')
    ).group_by(['m3','h']).agg(
        pl.col('temp_min').min().round(1).alias('temp_min_min'),
        pl.col('temp_min').median().round(1).alias('temp_min_med'),
        pl.col('temp_med').median().round(1).alias('temp_med_med'),
        pl.col('temp_max').median().round(1).alias('temp_max_med'),
        pl.col('temp_max').max().round(1).alias('temp_max_max')
    ).sort(['m3','h'])
    
    return({
        'df_cp_h':df_cp_h,
        'df_m3h':df_m3h,
        'df_mh':df_mh,
        'df_md':df_md.drop(['md','m3']),
        'df_m3':df_m3,
        'df_m':df_m,
        # 'df_cp_d':df_cp_d,
    })

data_output = create_output_cp(country,place,last_n_years=5,folder_data_raw='../data/raw')
data_output['df_md']

m,d,tmin_min,tmin_med,tavg_med,tmax_med,tmax_max,prcp_med,rain_med
i8,i8,f64,f64,f64,f64,f64,f64,f64
1,1,-3.0,5.7,6.8,8.6,15.7,3.5,1.0
1,2,-0.5,3.7,9.2,12.0,12.2,17.2,1.0
1,3,2.5,2.7,7.2,9.5,11.0,3.6,1.0
1,4,1.5,2.7,6.4,8.5,12.6,9.1,1.0
1,5,0.7,5.5,6.2,6.8,11.2,4.9,1.0
…,…,…,…,…,…,…,…,…
12,27,-0.4,2.0,4.6,6.5,9.2,0.1,1.0
12,28,1.5,6.3,8.1,9.8,11.0,3.3,0.5
12,29,2.6,5.5,9.0,10.7,12.0,8.9,1.0


In [282]:
df_cp_d.sort(['m','d','y']).tail(8)

time,tavg,tmin,tmax,prcp,wspd,pres,md,y,m,d,rain,m3
date,f64,f64,f64,f64,f64,f64,str,i32,i8,i8,i32,str
2020-12-30,3.0,0.7,5.2,null,12.6,999.5,"""12-30""",2020,12,30,null,"""12-3"""
2021-12-30,12.6,9.7,14.0,null,24.0,1013.1,"""12-30""",2021,12,30,null,"""12-3"""
2022-12-30,7.5,4.5,12.7,3.4,24.2,1004.4,"""12-30""",2022,12,30,1,"""12-3"""
2023-12-30,7.2,5.2,9.2,0.4,16.7,1010.1,"""12-30""",2023,12,30,1,"""12-3"""
2020-12-31,2.3,-0.7,3.2,null,7.1,1002.9,"""12-31""",2020,12,31,null,"""12-3"""
2021-12-31,12.8,12.0,13.7,null,23.7,1017.4,"""12-31""",2021,12,31,null,"""12-3"""
2022-12-31,13.6,11.0,15.7,20.0,24.0,1004.3,"""12-31""",2022,12,31,1,"""12-3"""
2023-12-31,8.6,7.7,9.2,4.7,24.0,997.7,"""12-31""",2023,12,31,1,"""12-3"""


In [ ]:
df_cp_d.group_by([])

In [203]:
# import os
# import polars as pl
# import pandas as pd
# from datetime import date

# country='egypte'
# place='sharm-el-sheikh'
# # country='spanje'
# # place='barcelona'
# # country='nederland'
# # place='ede'

# def loadfilteradd_cp(country,place,last_n_years=5,folder_data='./data/raw'):
#     # Load daily data:
#     fp_cp_d = os.path.join(folder_data,country,place,'weather_daily.csv')
#     df_cp_d = pl.read_csv(fp_cp_d,try_parse_dates=True).select(
#         ['time','tavg','tmin','tmax','prcp','wspd','pres']
#     ).cast(
#         {'time':pl.Date,
#          'tavg':pl.Float64,'tmin':pl.Float64,'tmax':pl.Float64,
#          'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64}
#     ).filter(
#         (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
#     ).with_columns(
#         pl.col('time').dt.strftime('%m-%d').alias('md'),
#         pl.col('time').dt.strftime('%m').alias('m_'),
#         pl.col('time').dt.month().alias('m'),
#         pl.when(pl.col("time").dt.day() > 20)
#             .then(3)
#             .when(pl.col("time").dt.day() > 10)
#             .then(2)
#             .otherwise(1)
#             .alias("m3_"),
#         pl.col('time').dt.day().alias('d'),
#         pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
#     ).with_columns(
#         pl.concat_str(['m_','m3_'],separator='-').alias('m3')
#     ).drop(['m_','m3_'])
#     # Load hourly data:
#     fp_cp_h = os.path.join(folder_data,country,place,'weather_hourly.csv')
#     df_cp_h = pl.read_csv(fp_cp_h,try_parse_dates=True).select(
#         ['time','temp','rhum','prcp','wspd','pres','coco']
#     ).cast(
#         {'time':pl.Datetime,
#          'temp':pl.Float64,'rhum':pl.Float64,
#          'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64,
#          'coco':pl.Float64
#         }
#     ).filter(
#         (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
#     ).with_columns(
#         pl.col('time').dt.strftime('%m-%d').alias('md'),
#         pl.col('time').dt.strftime('%m').alias('m_'),
#         pl.col('time').dt.month().alias('m'),
#         pl.col('time').dt.day().alias('d'),
#         pl.col('time').dt.hour().alias('h'),
#         pl.when(pl.col("time").dt.day() > 20)
#             .then(3)
#             .when(pl.col("time").dt.day() > 10)
#             .then(2)
#             .otherwise(1)
#             .alias("m3_"),
#         # pl.col('time').dt.day().alias('d'),
#         pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
#     ).with_columns(
#         pl.concat_str(['m_','m3_'],separator='-').alias('m3')
#     ).drop(['m_','m3_'])
#     return({'df_cp_h':df_cp_h,'df_cp_d':df_cp_d})

def agg_cp(country,place,last_n_years=5,folder_data='./data/raw'):
    # Load daily data:
    fp_cp_d = os.path.join(folder_data,country,place,'weather_daily.csv')
    df_cp_d = pl.read_csv(fp_cp_d,try_parse_dates=True).select(
        ['time','tavg','tmin','tmax','prcp','wspd','pres']
    ).cast(
        {'time':pl.Date,
         'tavg':pl.Float64,'tmin':pl.Float64,'tmax':pl.Float64,
         'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64}
    ).filter(
        (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
    ).with_columns(
        pl.col('time').dt.strftime('%m-%d').alias('md'),
        pl.col('time').dt.strftime('%m').alias('m_'),
        pl.col('time').dt.month().alias('m'),
        pl.when(pl.col("time").dt.day() > 20)
            .then(3)
            .when(pl.col("time").dt.day() > 10)
            .then(2)
            .otherwise(1)
            .alias("m3_"),
        pl.col('time').dt.day().alias('d'),
        pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
    ).with_columns(
        pl.concat_str(['m_','m3_'],separator='-').alias('m3')
    ).drop(['m_','m3_'])
    # Load hourly data:
    fp_cp_h = os.path.join(folder_data,country,place,'weather_hourly.csv')
    df_cp_h = pl.read_csv(fp_cp_h,try_parse_dates=True).select(
        ['time','temp','rhum','prcp','wspd','pres','coco']
    ).cast(
        {'time':pl.Datetime,
         'temp':pl.Float64,'rhum':pl.Float64,
         'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64,
         'coco':pl.Float64
        }
    ).filter(
        (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
    ).with_columns(
        pl.col('time').dt.strftime('%m-%d').alias('md'),
        pl.col('time').dt.strftime('%m').alias('m_'),
        pl.col('time').dt.month().alias('m'),
        pl.col('time').dt.day().alias('d'),
        pl.col('time').dt.hour().alias('h'),
        pl.when(pl.col("time").dt.day() > 20)
            .then(3)
            .when(pl.col("time").dt.day() > 10)
            .then(2)
            .otherwise(1)
            .alias("m3_"),
        # pl.col('time').dt.day().alias('d'),
        pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
    ).with_columns(
        pl.concat_str(['m_','m3_'],separator='-').alias('m3')
    ).drop(['m_','m3_'])

    # OUTPUTS:
    
    # /country/place/month
    df_md = df_cp_d.with_columns(
    #     pl.col('time').dt.strftime('%m-%d').alias('md'),
    #     pl.col('time').dt.strftime('%m').alias('m_'),
    #     pl.col('time').dt.month().alias('m'),
    #     pl.when(pl.col("time").dt.day() > 20)
    #         .then(3)
    #         .when(pl.col("time").dt.day() > 10)
    #         .then(2)
    #         .otherwise(1)
    #         .alias("m3_"),
    #     pl.col('time').dt.day().alias('d'),
    #     pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
    # ).with_columns(
    #     pl.concat_str(['m_','m3_'],separator='-').alias('m3')
    ).group_by(['md','m3','m','d']).agg(
        pl.col('tmin').min().round(1).alias('tmin_min'),
        pl.col('tmin').median().round(1).alias('tmin_med'),
        pl.col('tavg').median().round(1).alias('tavg_med'),
        pl.col('tmax').median().round(1).alias('tmax_med'),
        pl.col('tmax').max().round(1).alias('tmax_max'),
        (pl.col('rain').sum()/pl.count('rain')).round(2).alias('rain_pct'),
        (pl.col('prcp').sum()/pl.count('rain')).round().alias('rain_avg')
    ).sort('md')

    # /country/place
    df_m3 = df_md.group_by(['m3','m']).agg(
        pl.col('tmin_min').median().round(1).alias('tmin_min_med'),
        pl.col('tmin_med').median().round(1).alias('tmin_med_med'),
        pl.col('tavg_med').median().round(1).alias('tavg_med_med'),
        pl.col('tmax_med').median().round(1).alias('tmax_med_med'),
        pl.col('tmax_max').median().round(1).alias('tmax_max_med'),
        pl.col('rain_avg').sum().alias('rain_mm')
    ).sort('m3')

    
    df_m = df_md.group_by(['m']).agg(
        pl.col('tmin_min').median().round(1).alias('tmin_min_med'),
        pl.col('tmin_med').median().round(1).alias('tmin_med_med'),
        pl.col('tmax_med').median().round(1).alias('tmax_med_med'),
        pl.col('tmax_max').median().round(1).alias('tmax_max_med'),
        pl.col('rain_avg').sum().alias('rain_mm'),
        pl.col('rain_pct').sum().alias('rain_days')
    ).sort('m')


    # HIER GEBLEVEN:
    df_m3h = df_cp_h
    # Only keep relevant columns per df:
    
    return({
        'df_cp_h':df_cp_h,
        # 'df_m3h':df_m3h,
        'df_cp_d':df_cp_d,
        # 'df_md':df_md.drop(['m3','md','rain_pct']),
        # 'df_m3':df_m3,
        # 'df_m':df_m,
    })

agg_cp(country,place,last_n_years=5,folder_data='../data/raw')

{'df_cp_h': shape: (43_744, 13)
 ┌─────────────────────┬──────┬──────┬──────┬───┬─────┬─────┬──────┬──────┐
 │ time                ┆ temp ┆ rhum ┆ prcp ┆ … ┆ d   ┆ h   ┆ rain ┆ m3   │
 │ ---                 ┆ ---  ┆ ---  ┆ ---  ┆   ┆ --- ┆ --- ┆ ---  ┆ ---  │
 │ datetime[μs]        ┆ f64  ┆ f64  ┆ f64  ┆   ┆ i8  ┆ i8  ┆ i32  ┆ str  │
 ╞═════════════════════╪══════╪══════╪══════╪═══╪═════╪═════╪══════╪══════╡
 │ 2020-01-01 00:00:00 ┆ 12.2 ┆ 53.0 ┆ null ┆ … ┆ 1   ┆ 0   ┆ null ┆ 01-1 │
 │ 2020-01-01 01:00:00 ┆ 11.4 ┆ 55.0 ┆ null ┆ … ┆ 1   ┆ 1   ┆ null ┆ 01-1 │
 │ 2020-01-01 02:00:00 ┆ 10.5 ┆ 56.0 ┆ null ┆ … ┆ 1   ┆ 2   ┆ null ┆ 01-1 │
 │ 2020-01-01 03:00:00 ┆ 9.6  ┆ 59.0 ┆ null ┆ … ┆ 1   ┆ 3   ┆ null ┆ 01-1 │
 │ 2020-01-01 04:00:00 ┆ 10.1 ┆ 57.0 ┆ null ┆ … ┆ 1   ┆ 4   ┆ null ┆ 01-1 │
 │ …                   ┆ …    ┆ …    ┆ …    ┆ … ┆ …   ┆ …   ┆ …    ┆ …    │
 │ 2024-12-27 11:00:00 ┆ 15.2 ┆ 27.0 ┆ 0.0  ┆ … ┆ 27  ┆ 11  ┆ 0    ┆ 12-3 │
 │ 2024-12-27 12:00:00 ┆ 15.4 ┆ 26.0 ┆ 0.0  ┆ … ┆ 27  ┆ 

In [71]:
def combine_csvs_country(country,folder_data_processed='./data/processed'):
    dict_dfs_country={}
    folder_country = os.path.join(folder_data_processed,country)
    # Create a dictionary of data frames that combines similar csvs:
    for p in [x for x in os.listdir(folder_country) if os.path.isdir(os.path.join(folder_country,x))]:
        csvs_p = os.listdir(os.path.join(folder_country,p))
        for csv_p in csvs_p:
            # print(f'{p} {csv_p}')
            csv_pc = csv_p.replace('.csv','_country')
            df_p = pl.read_csv(os.path.join(folder_country,p,csv_p))
            df_p = df_p.with_columns(
                pl.lit(country).alias('country'),
                pl.lit(p).alias('place'),
            )
            # Debug:
            for col in ['rain_pct','prcp_avg','wspd_avg']:
                if col in df_p.columns:
                    df_p = df_p.cast({col:pl.Float64})
                
            if csv_pc in dict_dfs_country:
                dict_dfs_country[csv_pc]=pl.concat([dict_dfs_country[csv_pc],df_p])
            else:
                dict_dfs_country[csv_pc]=df_p
    # Sort the data frames:
    for k in dict_dfs_country:
        cols_sort = ['country','place']
        for col in ['m','m3','d','h','coco']:
            if col in dict_dfs_country[k].columns:
                cols_sort += [col]
        dict_dfs_country[k] = dict_dfs_country[k].sort(cols_sort)
    return(dict_dfs_country)

country='portugal'
dict_dfs_country = combine_csvs_country(country=country,folder_data_processed='../data/processed')
dict_dfs_country
# dictdfs_write_csv(dict_dfs_country,country,place=None,folder_data_processed='../data/processed')

figueira-da-foz df_mc.csv
figueira-da-foz df_mm3c.csv
figueira-da-foz df_mm3.csv
figueira-da-foz df_mm3d.csv
figueira-da-foz df_m.csv
figueira-da-foz df_mm3h.csv
figueira-da-foz df_mh.csv
lagos df_mc.csv
lagos df_mm3c.csv
lagos df_mm3.csv
lagos df_mm3d.csv
lagos df_m.csv
lagos df_mm3h.csv
lagos df_mh.csv
coimbra df_mc.csv
coimbra df_mm3c.csv
coimbra df_mm3.csv
coimbra df_mm3d.csv
coimbra df_m.csv
coimbra df_mm3h.csv
coimbra df_mh.csv
cascais df_mc.csv
cascais df_mm3c.csv
cascais df_mm3.csv
cascais df_mm3d.csv
cascais df_m.csv
cascais df_mm3h.csv
cascais df_mh.csv
lissabon df_mc.csv
lissabon df_mm3c.csv
lissabon df_mm3.csv
lissabon df_mm3d.csv
lissabon df_m.csv
lissabon df_mm3h.csv
lissabon df_mh.csv
carvoeiro df_mc.csv
carvoeiro df_mm3c.csv
carvoeiro df_mm3.csv
carvoeiro df_mm3d.csv
carvoeiro df_m.csv
carvoeiro df_mm3h.csv
carvoeiro df_mh.csv
nazare df_mc.csv
nazare df_mm3c.csv
nazare df_mm3.csv
nazare df_mm3d.csv
nazare df_m.csv
nazare df_mm3h.csv
nazare df_mh.csv
faro df_mc.csv
faro 

{'df_mc_country': shape: (2_652, 5)
 ┌─────┬──────┬──────────┬──────────┬───────────┐
 │ m   ┆ coco ┆ coco_pct ┆ country  ┆ place     │
 │ --- ┆ ---  ┆ ---      ┆ ---      ┆ ---       │
 │ i64 ┆ f64  ┆ f64      ┆ str      ┆ str       │
 ╞═════╪══════╪══════════╪══════════╪═══════════╡
 │ 1   ┆ 1.0  ┆ 0.412    ┆ portugal ┆ albufeira │
 │ 1   ┆ 2.0  ┆ 0.311    ┆ portugal ┆ albufeira │
 │ 1   ┆ 3.0  ┆ 0.183    ┆ portugal ┆ albufeira │
 │ 1   ┆ 4.0  ┆ 0.036    ┆ portugal ┆ albufeira │
 │ 1   ┆ 5.0  ┆ 0.013    ┆ portugal ┆ albufeira │
 │ …   ┆ …    ┆ …        ┆ …        ┆ …         │
 │ 12  ┆ 19.0 ┆ 0.0      ┆ portugal ┆ tavira    │
 │ 12  ┆ 23.0 ┆ 0.0      ┆ portugal ┆ tavira    │
 │ 12  ┆ 25.0 ┆ 0.0      ┆ portugal ┆ tavira    │
 │ 12  ┆ 26.0 ┆ 0.0      ┆ portugal ┆ tavira    │
 │ 12  ┆ 27.0 ┆ 0.0      ┆ portugal ┆ tavira    │
 └─────┴──────┴──────────┴──────────┴───────────┘,
 'df_mm3c_country': shape: (7_956, 6)
 ┌─────┬─────┬──────┬──────────┬──────────┬───────────┐
 │ m   ┆ m3  ┆ coco

In [70]:
df_p = pl.read_csv('../data/processed/indonesië/balikpapan/df_mm3d.csv')
for col in ['rain_pct']:
    if col in df_p.columns:
        df_p = df_p.cast({col:pl.Float64})

df_p

m,m3,d,tmin_avg,tavg_avg,tmax_avg,prcp_avg,wspd_avg,rain_pct
i64,i64,i64,f64,f64,f64,str,f64,f64
1,1,1,13.3,15.2,18.4,null,11.7,null
1,1,2,12.5,14.5,17.8,null,9.8,null
1,1,3,12.8,14.2,17.0,null,8.9,null
1,1,4,13.0,15.3,17.5,null,11.4,null
1,1,5,12.9,14.3,17.6,null,14.0,null
…,…,…,…,…,…,…,…,…
12,3,27,14.0,15.6,18.1,null,9.9,null
12,3,28,13.5,15.5,17.7,null,8.9,null
12,3,29,13.2,15.1,17.5,null,10.4,null


In [78]:

def combine_csvs_country(country,folder_data_processed='./data/processed'):
    dict_dfs_country={}
    folder_country = os.path.join(folder_data_processed,country)
    # Create a dictionary of data frames that combines similar csvs:
    for p in [x for x in os.listdir(folder_country) if os.path.isdir(os.path.join(folder_country,x))]:
        csvs_p = os.listdir(os.path.join(folder_country,p))
        for csv_p in csvs_p:
            # print(f'{p} {csv_p}')
            csv_pc = csv_p.replace('.csv','_country')
            df_p = pl.read_csv(os.path.join(folder_country,p,csv_p))
            df_p = df_p.with_columns(
                pl.lit(country).alias('country'),
                pl.lit(p).alias('place'),
            )
            # Debug:
            for col in ['rain_pct','prcp_avg','wspd_avg','coco','coco_pct']:
                if col in df_p.columns:
                    df_p = df_p.cast({col:pl.Float64})
            for col in ['m','m3','d','h','rain']:
                if col in df_p.columns:
                    df_p = df_p.cast({col:pl.Int8})
            if csv_pc in dict_dfs_country:
                print(f'{csv_pc} - {p}')
                dict_dfs_country[csv_pc]=pl.concat([dict_dfs_country[csv_pc],df_p])
            else:
                dict_dfs_country[csv_pc]=df_p
    # Sort the data frames:
    for k in dict_dfs_country:
        cols_sort = ['country','place']
        for col in ['m','m3','d','h','coco']:
            if col in dict_dfs_country[k].columns:
                cols_sort += [col]
        dict_dfs_country[k] = dict_dfs_country[k].sort(cols_sort)
    return(dict_dfs_country)

combine_csvs_country(country='indonesië',folder_data_processed='../data/processed')

df_mc_country - lombok
df_mm3c_country - lombok
df_mm3_country - lombok
df_mm3d_country - lombok
df_m_country - lombok
df_mm3h_country - lombok
df_mh_country - lombok
df_mc_country - balikpapan
df_mm3c_country - balikpapan
df_mm3_country - balikpapan
df_mm3d_country - balikpapan
df_m_country - balikpapan
df_mm3h_country - balikpapan
df_mh_country - balikpapan
df_mc_country - ubud
df_mm3c_country - ubud
df_mm3_country - ubud
df_mm3d_country - ubud
df_m_country - ubud
df_mm3h_country - ubud
df_mh_country - ubud
df_mc_country - yogyakarta
df_mm3c_country - yogyakarta
df_mm3_country - yogyakarta
df_mm3d_country - yogyakarta
df_m_country - yogyakarta
df_mm3h_country - yogyakarta
df_mh_country - yogyakarta
df_mc_country - sanur
df_mm3c_country - sanur
df_mm3_country - sanur
df_mm3d_country - sanur
df_m_country - sanur
df_mm3h_country - sanur
df_mh_country - sanur
df_mc_country - flores
df_mm3c_country - flores
df_mm3_country - flores
df_mm3d_country - flores
df_m_country - flores
df_mm3h_cou

{'df_mc_country': shape: (1_344, 5)
 ┌─────┬──────┬──────────┬───────────┬────────────┐
 │ m   ┆ coco ┆ coco_pct ┆ country   ┆ place      │
 │ --- ┆ ---  ┆ ---      ┆ ---       ┆ ---        │
 │ i8  ┆ f64  ┆ f64      ┆ str       ┆ str        │
 ╞═════╪══════╪══════════╪═══════════╪════════════╡
 │ 1   ┆ 1.0  ┆ 0.0      ┆ indonesië ┆ balikpapan │
 │ 1   ┆ 2.0  ┆ 0.012    ┆ indonesië ┆ balikpapan │
 │ 1   ┆ 3.0  ┆ 0.479    ┆ indonesië ┆ balikpapan │
 │ 1   ┆ 5.0  ┆ 0.039    ┆ indonesië ┆ balikpapan │
 │ 1   ┆ 7.0  ┆ 0.141    ┆ indonesië ┆ balikpapan │
 │ …   ┆ …    ┆ …        ┆ …         ┆ …          │
 │ 12  ┆ 9.0  ┆ 0.051    ┆ indonesië ┆ ubud       │
 │ 12  ┆ 17.0 ┆ 0.0      ┆ indonesië ┆ ubud       │
 │ 12  ┆ 18.0 ┆ 0.012    ┆ indonesië ┆ ubud       │
 │ 12  ┆ 25.0 ┆ 0.031    ┆ indonesië ┆ ubud       │
 │ 12  ┆ 27.0 ┆ 0.0      ┆ indonesië ┆ ubud       │
 └─────┴──────┴──────────┴───────────┴────────────┘,
 'df_mm3c_country': shape: (4_032, 6)
 ┌─────┬─────┬──────┬──────────┬─────────

In [77]:
pl.read_csv('../data/processed/indonesië/yogyakarta/df_mc.csv')

m,coco,coco_pct
str,str,str
